## [12-27] 流平衡写的是有问题的

## [12-30] 输出结果是遍历所有点不重复边的

## [12-30] 会输出 x[a,d] 这个无效弧 且这个弧度的值会趋于最大边界 虽然无影响 但是不知道是怎么形成的 

In [38]:
import gurobipy as gb
from gurobipy import*
import openpyxl  
import numpy as np
import math
import time
import copy
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import heapq
from collections import defaultdict
import os
import sys
import json
import collections


In [39]:
with open('wanna_range.json','r') as f:
    wanna_range = json.load(f)

In [40]:
# PARA
M = 10E5
E = 10E-2
ISSme = 1

In [41]:
def get_values(filename,id = 0,delme = 0):
    wb = openpyxl.load_workbook(filename)
    sheet_names = wb.sheetnames
    print(sheet_names)

    sheet = wb[sheet_names[id]]
    maxrow = sheet.max_row 
    maxcol = sheet.max_column 
    value = sheet.cell(row=2, column=1).value
    alldata = []

    for row in sheet.iter_rows(min_row=1, max_row=maxrow, min_col=1, max_col=maxcol): # type: ignore
        row_data = [cell.value for cell in row]
        alldata.append(row_data)
    station_name = alldata[0][1:]

    if delme:
        del alldata[0]
        for i in range(len(alldata)):
            del alldata[i][0]


    return alldata,station_name

In [42]:
def refineme(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] is None or data[i][j] == 0: 
                data[i][j] = M
    return data

In [43]:
def convme(data,title):
    degree = {}
    alldegree = 0
    for i,row in enumerate(data):
        
        degree[title[i]] = sum(row)
        alldegree = alldegree + sum(row)
    return degree,alldegree


In [44]:

def dfs_paths(graph, start, end, path, all_paths):
    path.append(start)
    if start == end:
        all_paths.append(path[:])
    else:
        for neighbor in graph[start]:
            if neighbor not in path:  # 防止重复访问
                dfs_paths(graph, neighbor, end, path, all_paths)
    path.pop()

In [45]:
def findme_maxdegreee(degree):
    max_degree = max(degree.values())

    max_degree_node = []
    for key in degree:
        if int(degree[key]) == int(max_degree):
            max_degree_node.append(key)
    return(max_degree,max_degree_node)

In [46]:
def picme(station_name,MyOD):
    plt.figure(figsize=(10, 6))
    MyOD = np.array(MyOD)
    plt.imshow(MyOD, cmap='Blues', aspect='auto', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xticks(ticks=np.arange(len(station_name)), labels=station_name, rotation=45)
    plt.yticks(ticks=np.arange(MyOD.shape[0]), labels=[f'Row {i+1}' for i in range(MyOD.shape[0])])
    plt.title('Heatmap of Matrix')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.tight_layout()
    plt.show()

In [47]:
MyOD,station_name = get_values(f'random_gen@{wanna_range}.xlsx',delme=1)

['Sheet1']


In [48]:
MyOD = refineme(MyOD)

In [49]:
station_name.insert(0,'d')
station_name.append('a')

In [50]:
# 设置插入的值
value_to_insert = M/20

# 示例数据


# 插入新列（在每行的开头和结尾都插入100）
for row in MyOD:
    row.insert(0, value_to_insert)  # 插入开头的列
    row.append(value_to_insert)     # 插入结尾的列

# 打印结果
print("After inserting columns:")
for row in MyOD:
    print(row)

print('\n')

# 插入新行（在开头和结尾插入100的行）
new_row = [value_to_insert for _ in range(len(MyOD[0]))]  # 创建一个独立的新行
MyOD.insert(0, new_row)  # 在开头插入新行
MyOD.append([value_to_insert for _ in range(len(MyOD[0]))])  # 在结尾插入独立的新行

# 打印结果
print("After inserting rows:")
for row in MyOD:
    print(row)


After inserting columns:
[50000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0

In [51]:
MyOD = copy.deepcopy(MyOD)

In [52]:
for i, row in enumerate(MyOD):
    for j, item in enumerate(row):
        # 第一行第一个元素
        if i == 0 and j == 0:

            MyOD[i][j] = M

        # 第一行最后一个元素
        elif i == 0 and j == len(row) - 1:
            MyOD[i][j] = M

        # 第一列或最后一列的元素
        elif j == 0 :
            MyOD[i][j] = M

        # 最后一行的元素
        elif i == len(MyOD) - 1:

            MyOD[i][j] = M


In [53]:
# 打印结果
for row in MyOD:
    print(row)

[1000000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 1000000.0]
[1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000

In [54]:
model1 = gb.Model()

In [55]:
x = {}
y = {}
z = {}
for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            x[i,j] = model1.addVar(vtype=gb.GRB.INTEGER,lb = 0,ub = len(MyOD)*len(MyOD),name = f'[x{station_name[i]},{station_name[j]}]')
            y[i,j] = model1.addVar(vtype=gb.GRB.BINARY,name = f'[y{station_name[i]},{station_name[j]}]') # 不重复
            z[i,j] = model1.addVar(vtype=gb.GRB.BINARY,name = f'[z{station_name[i]},{station_name[j]}]') # n -> 1 约束

In [56]:
for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i != j:
            model1.addConstr(x[i, j] <= M * z[i, j])  # 上界约束
            model1.addConstr(x[i, j] >= z[i, j])      # 下界约束

In [57]:
'''for i in range(0,len(MyOD)-1,1 ):
    for j in range(0,len(MyOD)-1,1 ):
        if i != j:
            model1.addConstr(x[i,j] >= M*(z[i,j]-1)) # 1+ -> 1

            model1.addConstr(x[i,j] <= ME*z[i,j] )'''

'for i in range(0,len(MyOD)-1,1 ):\n    for j in range(0,len(MyOD)-1,1 ):\n        if i != j:\n            model1.addConstr(x[i,j] >= M*(z[i,j]-1)) # 1+ -> 1\n\n            model1.addConstr(x[i,j] <= ME*z[i,j] )'

In [58]:
# flow consit
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1))== 
                 quicksum(x[j,len(MyOD)-1]  for j in range(1,len(MyOD)-1,1) ))

<gurobi.Constr *Awaiting Model Update*>

In [59]:
for j in range(1,len(MyOD)-1,1):
    model1.addConstr(quicksum(x[i,j] for i in range(len(MyOD)) if i!= j) 
                     == quicksum(x[j,k] for k in range(len(MyOD) )if k!= j))

In [60]:
# node must visit
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [61]:
model1.addConstr(quicksum(x[k,len(MyOD)-1] for k in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [62]:
for i in range(1,len(MyOD)-1,1):
    model1.addConstr(quicksum(x[k,i] for k in range(len(MyOD)) if k !=i) >= 1)


In [63]:
# path one direction
for i in range(0,len(MyOD)-1,1 ):
    for j in range(0,len(MyOD)-1,1 ):
        if i != j:
            model1.addConstr(x[i,j] <= M * y[i,j])
            model1.addConstr(x[j,i] <= M * (1-y[i,j]))

In [64]:
Z1 = quicksum(int(MyOD[i][j])*x[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)

In [65]:
#main
Z2 = quicksum(int(MyOD[0][j])*z[0,j]  for j in range(len(MyOD))if j != 0) + quicksum(int(MyOD[j][len(MyOD)-1])*z[j,len(MyOD)-1]  for j in range(len(MyOD)) if j != len(MyOD)-1) + quicksum(int(MyOD[i][j])*x[i,j] for i in range(1,len(MyOD),1 )for j in range(1,len(MyOD),1)if i != j)


In [66]:
Z3 = quicksum(int(MyOD[i][j])*z[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)

In [67]:
model1.setObjective(Z2, sense=gb.GRB.MINIMIZE)

In [68]:
model1.update()

In [69]:
model1.write("model.lp")

In [70]:
model1.setParam("IntFeasTol", 1e-9)  # 控制整数容差
model1.setParam("MIPGap", 1e-9)      # 控制混合整数问题的全局优化间隙


Set parameter IntFeasTol to value 1e-09
Set parameter MIPGap to value 1e-09


In [71]:
print('\n')

model1.setParam(gb.GRB.Param.TimeLimit, 3600*2)
model1.optimize()

try:
    if model1.ObjVal >= M*2:
        print(f'Model result of {model1.ObjVal:.2f}  exceed M:{M:.2f}')
        print('Result may out of node\\arc ability ')
    else:
        print(model1.ObjVal)
except Exception as e:
    print(e)
    print('\033[91m\nFatal! Model is INFEASIBLE.')

    if ISSme:
        model1.computeIIS()
        model1.write("modelv0.4.ilp")
    else:
        print('\033[5m\n ISS CHECK CANCEL \n\033[0m')



Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-14600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 41007 rows, 30906 columns and 112308 nonzeros
Model fingerprint: 0x2d040d71
Variable types: 0 continuous, 30906 integer (20604 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 2e+06]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+06]
Presolve removed 5 rows and 105 columns
Presolve time: 0.07s
Presolved: 41002 rows, 30801 columns, 102202 nonzeros
Variable types: 0 continuous, 30801 integer (20400 binary)
Found heuristic solution: objective 9.915000e+07
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective 5.010861e

In [72]:
#print(p[reflector[i], reflector[j],k].varName,p[reflector[i], reflector[j],k].x)

roadrec = []
mynode = [0,0]

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j :
            if x[i,j].x >= 0.1 :
                if MyOD[i][j] >= M:
                    print('OD Should not choose ',x[i,j].varName,'x',x[i,j].x,'z',z[i,j].x,'\n',x[i,j].x*MyOD[i][j],int(MyOD[i][j])*z[i,j].x)
                else:
                    print(x[i,j].varName,'x',x[i,j].x,'z',z[i,j].x,'\n',x[i,j].x*MyOD[i][j],int(MyOD[i][j])*z[i,j].x)
                    if i == 0:
                        mynode[0] = mynode[0]+1
                    if j == len(MyOD)-1:
                        mynode[1] = mynode[1]+1

                    roadrec.append([station_name[i],station_name[j],x[i,j].x,x[i,j].x*MyOD[i][j]])

[xd,V1] x 1.0 z 1.0 
 50000.0 50000.0
[xV1,V14] x 1.0 z 1.0 
 1.0 1.0
[xV2,V3] x 1.0 z 1.0 
 1.0 1.0
[xV3,V94] x 1.0 z 1.0 
 1.0 1.0
[xV4,V20] x 1.0 z 1.0 
 1.0 1.0
[xV5,V99] x 1.0 z 1.0 
 1.0 1.0
[xV6,V96] x 1.0 z 1.0 
 1.0 1.0
[xV7,V92] x 1.0 z 1.0 
 1.0 1.0
[xV8,V37] x 1.0 z 1.0 
 1.0 1.0
[xV9,V77] x 1.0 z 1.0 
 1.0 1.0
[xV10,V79] x 1.0 z 1.0 
 1.0 1.0
[xV11,V78] x 1.0 z 1.0 
 1.0 1.0
[xV12,V97] x 1.0 z 1.0 
 1.0 1.0
[xV13,a] x 1.0 z 1.0 
 50000.0 50000.0
[xV14,V88] x 1.0 z 1.0 
 1.0 1.0
[xV15,V17] x 1.0 z 1.0 
 1.0 1.0
[xV16,V73] x 1.0 z 1.0 
 1.0 1.0
[xV17,V74] x 1.0 z 1.0 
 1.0 1.0
[xV18,V40] x 1.0 z 1.0 
 1.0 1.0
[xV19,V27] x 1.0 z 1.0 
 1.0 1.0
[xV20,V53] x 1.0 z 1.0 
 1.0 1.0
[xV21,V35] x 1.0 z 1.0 
 1.0 1.0
[xV22,V45] x 1.0 z 1.0 
 1.0 1.0
[xV23,V54] x 1.0 z 1.0 
 1.0 1.0
[xV24,V67] x 1.0 z 1.0 
 1.0 1.0
[xV25,V52] x 1.0 z 1.0 
 1.0 1.0
[xV26,V90] x 1.0 z 1.0 
 1.0 1.0
[xV27,V65] x 1.0 z 1.0 
 1.0 1.0
[xV28,V34] x 1.0 z 1.0 
 1.0 1.0
[xV29,V87] x 1.0 z 1.0 
 1.0 1.0
[xV30,V63

In [73]:
MyOD[17][0]

1000000.0

In [74]:
# 构建邻接表


graph = defaultdict(list)
for start, end, _, _ in roadrec:
    graph[start].append(end)



# 寻找从 'd' 到 'a' 的所有路径
start_node, end_node = 'd', 'a'
all_paths = []
dfs_paths(graph, start_node, end_node, [], all_paths)
all_paths

[['d',
  'V1',
  'V14',
  'V88',
  'V46',
  'V48',
  'V72',
  'V23',
  'V54',
  'V30',
  'V63',
  'V70',
  'V47',
  'V57',
  'V58',
  'V8',
  'V37',
  'V93',
  'V75',
  'V66',
  'V19',
  'V27',
  'V65',
  'V9',
  'V77',
  'V100',
  'V2',
  'V3',
  'V94',
  'V31',
  'V21',
  'V35',
  'V4',
  'V20',
  'V53',
  'V82',
  'V44',
  'V95',
  'V28',
  'V34',
  'V64',
  'V69',
  'V36',
  'V6',
  'V96',
  'V68',
  'V50',
  'V16',
  'V73',
  'V18',
  'V40',
  'V7',
  'V92',
  'V49',
  'V25',
  'V52',
  'V76',
  'V15',
  'V17',
  'V74',
  'V29',
  'V87',
  'V89',
  'V91',
  'V71',
  'V24',
  'V67',
  'V56',
  'V98',
  'V42',
  'V13',
  'a']]